# **ATIVIDADE 3 - NAIVES BAYES**
---
---

Para esta atividade, devemos anotar a acurácia utiliando os algoritmos `MultinomialNB`, `GaussianNB` e `ComplementNB`, para as mesmas bases de dados que utilizamos nas atividades anteriores.

## **1.Importando as Bibliotecas**

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.naive_bayes import GaussianNB, MultinomialNB, ComplementNB
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler

## **2.Importando as Bases**

In [10]:
hog_128_16 = pd.read_csv('../datasets/hog_128_16.csv')
hog_128_20 = pd.read_csv('../datasets/hog_128_20.csv')
cnn_VGG16_AVG_128 = pd.read_csv('../datasets/cnn_VGG16_AVG_128.csv')
cnn_VGG19_AVG_128 = pd.read_csv('../datasets/cnn_VGG19_AVG_128.csv')
cnn_VGG16_MAX_128 = pd.read_csv('../datasets/cnn_VGG16_MAX_128.csv')
cnn_VGG19_MAX_128 = pd.read_csv('../datasets/cnn_VGG19_MAX_128.csv')
hog_128_16_PCA = pd.read_csv('../datasets/hog_128_16_PCA.csv')
hog_128_20_PCA = pd.read_csv('../datasets/hog_128_20_PCA.csv')
cnn_VGG16_AVG_128_PCA = pd.read_csv('../datasets/cnn_VGG16_AVG_128_PCA.csv')
cnn_VGG19_AVG_128_PCA= pd.read_csv('../datasets/cnn_VGG19_AVG_128_PCA.csv')
cnn_VGG16_MAX_128_PCA= pd.read_csv('../datasets/cnn_VGG16_MAX_128_PCA.csv')
cnn_VGG19_MAX_128_PCA = pd.read_csv('../datasets/cnn_VGG19_MAX_128_PCA.csv')


## **3.Códigos**

#### 3.1.Instanciando Dict com Todos os DataFrames

Esta estapa serve para conseguirmos passar o nome do dataset na hora de criarmos o DataFrame com as acurácias.

In [11]:
dataframes = {
    'hog_128_16': hog_128_16,
    'hog_128_20': hog_128_20,
    'cnn_VGG16_AVG_128': cnn_VGG16_AVG_128,
    'cnn_VGG19_AVG_128': cnn_VGG19_AVG_128,
    'cnn_VGG16_MAX_128': cnn_VGG16_MAX_128,
    'cnn_VGG19_MAX_128': cnn_VGG19_MAX_128,
    'hog_128_16_PCA': hog_128_16_PCA,
    'hog_128_20_PCA': hog_128_20_PCA,
    'cnn_VGG16_AVG_128_PCA': cnn_VGG16_AVG_128_PCA,
    'cnn_VGG19_AVG_128_PCA': cnn_VGG19_AVG_128_PCA,
    'cnn_VGG16_MAX_128_PCA': cnn_VGG16_MAX_128_PCA,
    'cnn_VGG19_MAX_128_PCA': cnn_VGG19_MAX_128_PCA
}

#### 3.2.Configuração do Dataframe das Acurácias 

In [12]:
multi_index = []
for name in dataframes.keys():
    multi_index.extend([(name, '70/30'), (name, '10-fold CV')])
accuracy_df = pd.DataFrame(index=pd.MultiIndex.from_tuples(multi_index), 
                           columns=['GaussianNB', 'MultinomialNB', 'ComplementNB'])

#### 3.3.Inicializando os Modelos

In [13]:
models = {
    'GaussianNB': GaussianNB(priors=None, var_smoothing=1e-09),
    'MultinomialNB': MultinomialNB(fit_prior=True, alpha=1.0),
    'ComplementNB': ComplementNB(alpha=1.0, force_alpha=True, fit_prior=True)
}

Algo que teremos que fazer é normalizar os dados das bases com PCA, pois tanto o MultinomialNB e ComplementNB não funcionam com valores negativos. Normalizando, iremos manter a distribuição dos dados mas iremos setar seus valores com base na média igual a zero.

#### 3.4.Loop

Loop que irá realizar para cada uma das bases:
* Divisão holdout 70/30 e k-fold (k=10)
* Treinamento para todos os modelos
* Avaliação para as repectivas base de testes
* Salva a suas respectivas acurácias no DataFrame

In [14]:
# Loop para processar cada dataset
for name, df in dataframes.items():
    print(f"Processando dataset: {name}")
    
    X = df.iloc[:, 1:]  # Features
    y = df.iloc[:, 0]   # Target
    
    # Verificar valores negativos no dataset
    contains_negatives = (X < 0).any().any()
    
    # Holdout 70/30
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    holdout_accuracies = {}
    
    for model_name, model in models.items():
        # Tratamento para MultinomialNB e ComplementNB com valores negativos
        if model_name in ['MultinomialNB', 'ComplementNB']:
            # Normalizar para garantir compatibilidade
            scaler = MinMaxScaler()
            X_train = scaler.fit_transform(X_train)
            X_test = scaler.transform(X_test)
            print(f"Normalização aplicada para {model_name} no dataset {name}.")
        
        # Treinamento do modelo
        try:
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            acc = accuracy_score(y_test, y_pred)
            holdout_accuracies[model_name] = acc
        except ValueError as e:
            print(f"Erro ao treinar {model_name} no dataset {name}: {e}")
            holdout_accuracies[model_name] = np.nan
    
    # Salvando os resultados do Holdout
    for model_name, acc in holdout_accuracies.items():
        accuracy_df.loc[(name, '70/30'), model_name] = acc
    
    # KFold com k=10
    kf = KFold(n_splits=10, shuffle=True, random_state=42)
    kfold_accuracies = {model_name: [] for model_name in models.keys()}
    
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        for model_name, model in models.items():
            # Tratamento para MultinomialNB e ComplementNB com valores negativos
            if model_name in ['MultinomialNB', 'ComplementNB']:
                # Normalizar para garantir compatibilidade
                scaler = MinMaxScaler()
                X_train = scaler.fit_transform(X_train)
                X_test = scaler.transform(X_test)
            
            # Treinamento do modelo
            try:
                model.fit(X_train, y_train)
                y_pred = model.predict(X_test)
                acc = accuracy_score(y_test, y_pred)
                kfold_accuracies[model_name].append(acc)
            except ValueError as e:
                print(f"Erro ao treinar {model_name} no dataset {name} (KFold): {e}")
                kfold_accuracies[model_name].append(np.nan)
    
    # Calculando a média das acurácias do KFold
    for model_name, acc_list in kfold_accuracies.items():
        if acc_list:  # Garantir que não seja vazio (caso MultinomialNB ou ComplementNB tenha sido ignorado)
            accuracy_df.loc[(name, '10-fold CV'), model_name] = np.nanmean(acc_list)

Processando dataset: hog_128_16
Normalização aplicada para MultinomialNB no dataset hog_128_16.
Normalização aplicada para ComplementNB no dataset hog_128_16.
Processando dataset: hog_128_20
Normalização aplicada para MultinomialNB no dataset hog_128_20.
Normalização aplicada para ComplementNB no dataset hog_128_20.
Processando dataset: cnn_VGG16_AVG_128
Normalização aplicada para MultinomialNB no dataset cnn_VGG16_AVG_128.
Normalização aplicada para ComplementNB no dataset cnn_VGG16_AVG_128.
Processando dataset: cnn_VGG19_AVG_128
Normalização aplicada para MultinomialNB no dataset cnn_VGG19_AVG_128.
Normalização aplicada para ComplementNB no dataset cnn_VGG19_AVG_128.
Processando dataset: cnn_VGG16_MAX_128
Normalização aplicada para MultinomialNB no dataset cnn_VGG16_MAX_128.
Normalização aplicada para ComplementNB no dataset cnn_VGG16_MAX_128.
Processando dataset: cnn_VGG19_MAX_128
Normalização aplicada para MultinomialNB no dataset cnn_VGG19_MAX_128.
Normalização aplicada para Compl

In [15]:
accuracy_df

GaussianNB MultinomialNB ComplementNB
hog_128_16            70/30        0.658333      0.645833     0.645833
                      10-fold CV   0.676614      0.645538     0.646788
hog_128_20            70/30        0.683333      0.641667     0.645833
                      10-fold CV   0.665491      0.646772     0.646772
cnn_VGG16_AVG_128     70/30        0.495833      0.583333       0.5875
                      10-fold CV   0.508687      0.622785     0.631551
cnn_VGG19_AVG_128     70/30        0.483333      0.579167     0.566667
                      10-fold CV   0.521218      0.601566     0.606535
cnn_VGG16_MAX_128     70/30        0.495833      0.604167          0.6
                      10-fold CV   0.506203      0.621472     0.626487
cnn_VGG19_MAX_128     70/30        0.479167      0.504167       0.5625
                      10-fold CV   0.521282      0.599051     0.615316
hog_128_16_PCA        70/30            0.65        0.5625     0.620833
                      10-fold CV   0.651756      0.618038     0.650554
hog_128_20_PCA        70/30        0.641667        0.5875         0.65
                      10-fold CV   0.635459      0.602943     0.635443
cnn_VGG16_AVG_128_PCA 70/30          0.5875          0.55     0.579167
                      10-fold CV   0.561313      0.587785     0.621535
cnn_VGG19_AVG_128_PCA 70/30          0.5375      0.479167     0.579167
                      10-fold CV   0.590158      0.530206     0.602785
cnn_VGG16_MAX_128_PCA 70/30          0.5375      0.541667     0.541667
                      10-fold CV    0.54625      0.536535     0.600237
cnn_VGG19_MAX_128_PCA 70/30        0.508333      0.483333     0.554167
                      10-fold CV   0.557595      0.511345     0.565222

#### 3.5.Salvando em Excel

In [16]:
arquivo_excel = "acuracias_naive_bayes.xlsx"
accuracy_df.to_excel(arquivo_excel, sheet_name="Acurácias")
print(f"Arquivo salvo como '{arquivo_excel}'.")

Arquivo salvo como 'acuracias_naive_bayes.xlsx'.


## **4.Conclusões**

* O melhor método foi o ComplementNB.
* Todas as bases PCA foram piores do que as suas versões originais.